<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/add_new_tokens_to_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 言語モデルに新しいトークンを追加する方法

### 1. トークナイザとモデルを読み込む

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

* 語彙サイズを確認する。

In [2]:
len(tokenizer)

32000

### 2. 新しいトークンを追加する

In [3]:
num_added_toks = tokenizer.add_tokens(["bin_tok1", "bin_tok2", "bin_tok3", "bin_tok4", "bin_tok5"])
print("We have added", num_added_toks, "tokens")

We have added 5 tokens


* 語彙サイズを確認する。
  * 追加したトークンの数だけ、語彙数が増加している。

In [4]:
len(tokenizer)

32005

### 3. モデルの側の埋め込み行列のサイズを拡張する

* 拡張後の埋め込み行列のサイズが表示される。

In [5]:
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32005, 1024)

### 4. 新たに追加したトークンの埋め込みベクトルを初期化する

* copilotが薦めてきた初期化方法は、以下の通り。
  * 元の語彙の埋め込みベクトルの平均で初期化している。

In [6]:
model.transformer.wte.weight.data[-num_added_toks:] = model.transformer.wte.weight.data[:-num_added_toks].mean(dim=0)

In [7]:
model.transformer.wte.weight.data[-num_added_toks:]

tensor([[ 0.0135,  0.0274,  0.0002,  ...,  0.0326,  0.0012, -0.0153],
        [ 0.0135,  0.0274,  0.0002,  ...,  0.0326,  0.0012, -0.0153],
        [ 0.0135,  0.0274,  0.0002,  ...,  0.0326,  0.0012, -0.0153],
        [ 0.0135,  0.0274,  0.0002,  ...,  0.0326,  0.0012, -0.0153],
        [ 0.0135,  0.0274,  0.0002,  ...,  0.0326,  0.0012, -0.0153]])

In [8]:
model.transformer.wte.weight.data[:10]

tensor([[ 0.0607, -0.0704, -0.0182,  ..., -0.0844,  0.0696,  0.0853],
        [ 0.0066, -0.0474,  0.0583,  ..., -0.1230,  0.0563,  0.1173],
        [-0.0996,  0.0054, -0.1092,  ..., -0.0744,  0.0342,  0.0834],
        ...,
        [ 0.0332,  0.0826, -0.0331,  ..., -0.0969, -0.0697, -0.0048],
        [ 0.0522, -0.0911, -0.0006,  ...,  0.0686, -0.0153, -0.0308],
        [ 0.0159, -0.0321, -0.0277,  ..., -0.0347,  0.0517,  0.0813]])